In [55]:
import numpy as np
from scipy import linalg
from ase import Atoms
from ase.io import write

In [56]:
# Lattice parameter, Copper  
a_lat = 3.615  # [Angstroms]

# Cubic fractional basis, in rows. 
original_basis = np.array([
    [0.0, 0.0, 0.0],
    [0.5, 0.5, 0.0],
    [0.0, 0.5, 0.5],
    [0.5, 0.0, 0.5]
])

# Lattice vectors, in columns, or rows, its same here... but we will have to make sure all vectors should are column wise. 
original_lattice_vectors = np.eye(3) * a_lat

print(f"the original lattice vectors and basis vectors \n {original_lattice_vectors} \n\n  {original_basis}")

the original lattice vectors and basis vectors 
 [[3.615 0.    0.   ]
 [0.    3.615 0.   ]
 [0.    0.    3.615]] 

  [[0.  0.  0. ]
 [0.5 0.5 0. ]
 [0.  0.5 0.5]
 [0.5 0.  0.5]]


In [57]:
# New axes directions - new setting! we want the new lattice to be orthonormal and oriented along x y, z like so:
#x_new = np.array([0.5,   -0.5,    0]) 
#y_new = np.array([0.5,    0.5,   -1])
#z_new = np.array([1,    1 ,   1])

In [58]:
# New axes directions - new setting! we want the new lattice to be orthonormal and oriented along x y, z like so:
#x_new = np.array([1,   1,    0]) 
#y_new = np.array([1,    -1,   0])
#z_new = np.array([0,    0 ,   1])

In [59]:
# New axes directions - new setting! we want the new lattice to be orthonormal and oriented along x y, z like so:
#x_new = np.array([0.5,   0.5,    0]) 
#y_new = np.array([0.5,   -0.5,   0])
#z_new = np.array([0,    0 ,   1])

In [60]:
# New axes directions - new setting! we want the new lattice to be orthonormal and oriented along x y, z like so:
#x_new = np.array([1,   -1,    0]) 
#y_new = np.array([1,   1,   -2])
#z_new = np.array([1,   1 ,   1])

In [61]:
# New axes directions - new setting! we want the new lattice to be orthonormal and oriented along x y, z like so:
x_new = np.array([0.5,   -0.5,    0]) 
y_new = np.array([0.5,   0.5,   -1])
z_new = np.array([1,   1 ,   1])

In [62]:
# New axes directions - new setting! we want the new lattice to be orthonormal and oriented along x y, z like so:
#x_new = np.array([2, -2, 0]) 
#y_new = np.array([3, 3, -6])
#z_new = np.array([1,   1 ,   1])

In [63]:
# New axes directions - new setting! we want the new lattice to be orthonormal and oriented along x y, z like so:
#x_new = np.array([0.5,   -0.5,    0]) 
#y_new = np.array([0.5,   0.5,   -1])
#z_new = np.array([0.5,   0.5 ,   0.5])

In [64]:
# The rotation matrix is simply the one with each column a vector of the new lattice. Why?
rot_matrix = np.column_stack([x_new, y_new, z_new])

In [65]:
print(rot_matrix) # Note: This is 

[[ 0.5  0.5  1. ]
 [-0.5  0.5  1. ]
 [ 0.  -1.   1. ]]


In [66]:
# Calculate original lattice vectors in the rotated coordinate system! not x_new etc. 
# Read them column-wise

new_lattice_vectors = original_lattice_vectors @ rot_matrix 
print(new_lattice_vectors) # in original cartesian

[[ 1.8075  1.8075  3.615 ]
 [-1.8075  1.8075  3.615 ]
 [ 0.     -3.615   3.615 ]]


In [67]:
original_basis_cart=original_lattice_vectors @ original_basis.T # get the real coordinates basis vectors 
# i.e., according to the original cartesian coordinates. 
print(original_basis_cart)

[[0.     1.8075 0.     1.8075]
 [0.     1.8075 1.8075 0.    ]
 [0.     0.     1.8075 1.8075]]


In [68]:
A=linalg.inv(original_lattice_vectors) # the inverse of the lattice vectors matrix can convert a cartesian to lattice fractional vectors

In [69]:
A @ (original_basis_cart) # convert back to lattice coordinates using the original basis. This is just a check.

array([[0. , 0.5, 0. , 0.5],
       [0. , 0.5, 0.5, 0. ],
       [0. , 0. , 0.5, 0.5]])

In [70]:
B=linalg.inv(new_lattice_vectors) # similarly, will be used to convert from cart into the fractional units of the new lattice vectors. 

In [71]:
print(B)

[[ 2.76625173e-01 -2.76625173e-01 -3.17123443e-17]
 [ 9.22083910e-02  9.22083910e-02 -1.84416782e-01]
 [ 9.22083910e-02  9.22083910e-02  9.22083910e-02]]


In [72]:
new_basis=B @ original_basis_cart  
# Note: These are not all atoms if the new unit cell is larger! 
# so we need to find the other ones in teh same unit cell.

In [73]:
print(new_basis)

[[ 0.00000000e+00  1.52177141e-16 -5.00000000e-01  5.00000000e-01]
 [ 0.00000000e+00  3.33333333e-01 -1.66666667e-01 -1.66666667e-01]
 [ 0.00000000e+00  3.33333333e-01  3.33333333e-01  3.33333333e-01]]


In [74]:
# Let us find all new fractional coordinates, we use the original setting as a start, and cover a small portion in 
# cartesian space, then convert each to the new lattice vectors (still based on the same original cartesian system), 
# and finally take periodic bioundary conditions into account, i.e., we remove any atoms not in the original cell. 

In [75]:
point_grid = np.array([[i, j, k] for i in range(-10, 10) for j in range(-10, 10) for k in range(-10, 10)])

In [76]:
size_new_basis = 0 # number of basis atoms in the new system 
new_basis=[] # the new basis in fractional coordinated with respect to the new lattice vectors. 

for shift in point_grid:  # this is a grtid of points. 
    for atom in original_basis:
        new_atom_frac = atom + shift
        cart_atom = original_lattice_vectors @ new_atom_frac
        new_atom_frac_new_lattice = B @ cart_atom
        test=np.all(new_atom_frac_new_lattice >= -1e-6) and np.all(new_atom_frac_new_lattice < 1 - 1e-6) # boundary conditions
        if (test):
            print(f"shift={shift}, atom={atom}, new_atom_frac={new_atom_frac}, cart_atom={cart_atom}, \n new_atom_frac_new_lattice={new_atom_frac_new_lattice}")
            size_new_basis += 1
            new_basis.append(new_atom_frac_new_lattice)
print(f"size_new_basis={size_new_basis}")
new_basis = np.array(new_basis)

# Remove duplicates
new_basis = np.unique(np.round(new_basis, decimals=6), axis=0)


print("New lattice vectors (columns, in Å), rotated such as a1 // X, a2//y, a3 //z (new x, y, z):")

a1=linalg.norm(new_lattice_vectors[:,0])
a2=linalg.norm(new_lattice_vectors[:,1])
a3=linalg.norm(new_lattice_vectors[:,2])
new_lattice_xyz = np.diag([a1, a2, a3])

print(f"\n new_lattice_vectors = \n {new_lattice_vectors}, \n, new_lattice_xyz=\n{new_lattice_xyz}, \n new lattice normalised by a_lat=\n{new_lattice_xyz/a_lat}")
print("\nNew basis atom positions (fractional coordinates) in new system:")
for i, atom in enumerate(new_basis):
    print(f"Atom {i+1}: {atom}")

shift=[ 0  0 -1], atom=[0.5 0.  0.5], new_atom_frac=[ 0.5  0.  -0.5], cart_atom=[ 1.8075  0.     -1.8075], 
 new_atom_frac_new_lattice=[ 5.00000000e-01  5.00000000e-01 -5.67346231e-17]
shift=[0 0 0], atom=[0. 0. 0.], new_atom_frac=[0. 0. 0.], cart_atom=[0. 0. 0.], 
 new_atom_frac_new_lattice=[0. 0. 0.]
shift=[0 0 0], atom=[0.5 0.5 0. ], new_atom_frac=[0.5 0.5 0. ], cart_atom=[1.8075 1.8075 0.    ], 
 new_atom_frac_new_lattice=[1.52177141e-16 3.33333333e-01 3.33333333e-01]
shift=[ 1  0 -1], atom=[0.  0.5 0.5], new_atom_frac=[ 1.   0.5 -0.5], cart_atom=[ 3.615   1.8075 -1.8075], 
 new_atom_frac_new_lattice=[0.5        0.83333333 0.33333333]
shift=[1 0 0], atom=[0.  0.5 0.5], new_atom_frac=[1.  0.5 0.5], cart_atom=[3.615  1.8075 1.8075], 
 new_atom_frac_new_lattice=[0.5        0.16666667 0.66666667]
shift=[1 1 0], atom=[0. 0. 0.], new_atom_frac=[1. 1. 0.], cart_atom=[3.615 3.615 0.   ], 
 new_atom_frac_new_lattice=[3.04354283e-16 6.66666667e-01 6.66666667e-01]
size_new_basis=6
New lattice

In [77]:
# testing
# calculate the volume per atoms, which should be the same as the original.


# duplicate and visualise 


# calculate the RDF? 


In [78]:
# create a vasp input file, though later we could directly put this into an Atoms object


In [79]:
symbol = 'Cu'
symbols = [symbol] * len(new_basis)

print(len(new_basis))

6


In [80]:
# the new lattice vectors should be 1 0 0 , 0 1 0 , and 0 0 1, now, but with length given by the one of the lattice vectors in the original system:

In [81]:
new_atoms=Atoms(symbols=symbols, cell=new_lattice_xyz, pbc=True)

In [82]:
new_atoms.set_scaled_positions(new_basis)

In [83]:
new_atoms


Atoms(symbols='Cu6', pbc=True, cell=[2.5561910139893693, 4.427452710080595, 6.261363669361492])

In [84]:
write('POSCAR', new_atoms, format='vasp')